In [1]:
import os
import sys

import numpy as np
import tensorflow as tf

sys.path.insert(0, "/home/anonymous/Desktop/eeg_sampling/modeling")
from dataio import readdata, readlabels, writedata

In [2]:
#Read training
train_dataset = readdata("./curated/train_dataset")
train_labels = readlabels("./curated/train_labels")
#Read validation
valid_dataset = readdata("./curated/valid_dataset")
valid_labels = readlabels("./curated/valid_labels")
#Read test
test_dataset = readdata("./curated/test_dataset")
test_labels = readlabels("./curated/test_labels")

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (105, 1130, 8) (105,)
Validation: (21, 1130, 8) (21,)
Testing: (26, 1130, 8) (26,)


In [3]:
n_input = 8*1130 # EEG data input (8 channels * 1130 sample points)
n_classes = 5 # EEG total classes ("nothing", "up", "down", "left", "right")

In [4]:
def onehot(data, n_classes):
    b = np.zeros((data.size, n_classes))
    b[np.arange(data.size), data] = 1
    return(b)

train_dataset.shape = (train_dataset.shape[0], n_input)
valid_dataset.shape = (valid_dataset.shape[0], n_input)
test_dataset.shape = (test_dataset.shape[0], n_input)
train_labels = onehot(train_labels, n_classes)
valid_labels = onehot(valid_labels, n_classes)
test_labels = onehot(test_labels, n_classes)

In [62]:
# Parameters
learning_rate = 0.0001
training_epochs = 20
display_step = 2

# Network Parameters
n_hidden_1 = 3000 # 1st layer num features
n_hidden_2 = 5000 # 2nd layer num features
n_hidden_3 = 3000 # 2nd layer num features

In [63]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights["h1"]), _biases["b1"]))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights["h2"]), _biases["b2"]))
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, _weights["h3"]), _biases["b3"]))
    return tf.matmul(layer_3, _weights["out"]) + _biases["out"]

# Store layers weight & bias
weights = {
    "h1": tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    "h2": tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    "h3": tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    "out": tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}
biases = {
    "b1": tf.Variable(tf.random_normal([n_hidden_1])),
    "b2": tf.Variable(tf.random_normal([n_hidden_2])),
    "b3": tf.Variable(tf.random_normal([n_hidden_3])),
    "out": tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) # Softmax loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) # Adam Optimizer

In [64]:
# Initializing the variables
init = tf.initialize_all_variables()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        sess.run(optimizer, feed_dict={x: train_dataset, y: train_labels})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: train_dataset, y: train_labels})/train_dataset.size
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: test_dataset, y: test_labels}))

Epoch: 0001 cost= 0.375289224
Epoch: 0003 cost= 0.389421223
Epoch: 0005 cost= 0.544387214
Epoch: 0007 cost= 0.254830071
Epoch: 0009 cost= 0.219137975
Epoch: 0011 cost= 0.040999933
Epoch: 0013 cost= 0.000637941
Epoch: 0015 cost= 0.000000000
Epoch: 0017 cost= 0.000000000
Epoch: 0019 cost= 0.000000000
Optimization Finished!
Accuracy: 0.346154
